# Redis Caching

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd 
import numpy as np 
from urllib.request import urlopen
import redis
import json
import pysolr
import requests

In [3]:
pd.set_option('display.max_colwidth', -1)

In [4]:
site = "http://awesome-hw.sdsc.edu:8983/solr/#/student-news/query"

## Queries

In [1]:
def solr_query(string):
    """Function that takes in string of Queries and formats into solr query JSON
    
    Args: 
        param1(str): string of space separated queries
        ex1: "trump"
        ex2: "trump biden"
        
    Returns: 
        string: link of solr query output
    """
    
    link = 'http://awesome-hw.sdsc.edu:8983/solr/student-news/select?q='
    column = 'title%3A'
    space = '%20AND%20'
    searches = string.split(" ")
    multiple = ''
    
    #If there is only one search 
    if len(searches) == 1:
        link = link + column + searches[0]
    #Multiple searches to concatenate
    else:
        for i in range(1, len(searches)):
            multiple = multiple + space + column + searches[i]
        link = link + column + searches[0] + multiple
        
    return link

### (1) - Trump and Taxes

In [14]:
q1j = requests.get('http://awesome-hw.sdsc.edu:8983/solr/student-news/select?q=title%3A%22trump%22%20AND%20title%3A%22taxes%22')
out1 = q1j.json()

In [15]:
out1

{'responseHeader': {'status': 0,
  'QTime': 1,
  'params': {'q': 'title:"trump" AND title:"taxes"'}},
 'response': {'numFound': 222,
  'start': 0,
  'docs': [{'src': 'http://www.nytimes.com/pages/todayspaper/index.html',
    'title': 'Opinion | Trump’s Tax Avoidance Is a Tax on the Rest of Us',
    'news': 'We don’t know if Donald Trump’s recent federal income tax returns indicate illegal activity. But even if the $750 income tax bills he paid in each of the first two years of his presidency turn out to have been calculated in compliance with the U.S. Internal Revenue Code, he has enjoyed the benefits of the society we all contribute to build, then skipped out on his share of the bill — a sort of dine-and-dash on his fellow Americans.\n\nWhat many Americans may not realize is how common this is among the country’s top earners, and how much it costs the rest of us. Sixteen years ago, the former I.R.S. commissioner Charles Rossotti estimated that so many wealthy Americans don’t pay their

### (2) - Millions and COVID 

In [13]:
q2j = requests.get("http://awesome-hw.sdsc.edu:8983/solr/student-news/select?q=title%3A%22millions%22%20AND%20title%3A%22covid%22%20AND%20title%3A%22deaths%22")
out2 = q2j.json()

### (3) - Biden and Trump on Election Day

In [10]:
q3j = requests.get("http://awesome-hw.sdsc.edu:8983/solr/student-news/select?q=title%3A%22Biden%22%20AND%20title%3A%22Trump%22%20AND%20publishdate%3A%222020-11-03T00%3A00%3A00Z%22")
out3 = q3j.json()

## Implementing a Redis Cache

In [6]:
def format_response(response):
    out = {}
    out['title'] = response['title']
    out['publishdate'] = response['publishdate']
    return out

### Query 1

In [19]:
r1 = redis.Redis(host='my-redis', port=6379, db=0)

In [20]:
output1 = list(map(format_response, out1['response']['docs']))
output1

[{'title': 'Opinion | Trump’s Tax Avoidance Is a Tax on the Rest of Us',
  'publishdate': '2020-10-01T00:00:00Z'},
 {'title': 'Opinion | Trump’s Tax Avoidance Is a Tax on the Rest of Us',
  'publishdate': '2020-10-01T00:00:00Z'},
 {'title': 'Opinion | Trump’s Tax Avoidance Is a Tax on the Rest of Us',
  'publishdate': '2020-10-01T00:00:00Z'},
 {'title': 'Opinion | Trump’s Tax Avoidance Is a Tax on the Rest of Us',
  'publishdate': '2020-10-01T00:00:00Z'},
 {'title': 'Opinion | Trump’s Tax Avoidance Is a Tax on the Rest of Us',
  'publishdate': '2020-10-01T00:00:00Z'},
 {'title': 'Opinion | Trump’s Tax Avoidance Is a Tax on the Rest of Us',
  'publishdate': '2020-10-01T00:00:00Z'},
 {'title': 'Opinion | Trump’s Tax Avoidance Is a Tax on the Rest of Us',
  'publishdate': '2020-10-01T00:00:00Z'},
 {'title': 'Brooke Harrington: Trump’s tax avoidance is a tax on the rest of us',
  'publishdate': '2020-10-03T00:00:00Z'},
 {'title': 'Trump Tax-Return Report Fuels Democrats’ Drive to Tax the R

In [33]:
for out in output1:
    r1.append(out['publishdate'], out['title'])

In [34]:
pipe = r1.pipeline()
pipe.get('2020-10-01T00:00:00Z')
pipe.execute()

[b'Opinion | Trump\xe2\x80\x99s Tax Avoidance Is a Tax on the Rest of UsOpinion | Trump\xe2\x80\x99s Tax Avoidance Is a Tax on the Rest of UsOpinion | Trump\xe2\x80\x99s Tax Avoidance Is a Tax on the Rest of UsOpinion | Trump\xe2\x80\x99s Tax Avoidance Is a Tax on the Rest of UsOpinion | Trump\xe2\x80\x99s Tax Avoidance Is a Tax on the Rest of UsOpinion | Trump\xe2\x80\x99s Tax Avoidance Is a Tax on the Rest of UsOpinion | Trump\xe2\x80\x99s Tax Avoidance Is a Tax on the Rest of UsTrump Tax-Return Report Fuels Democrats\xe2\x80\x99 Drive to Tax the RichOpinion | Trump\xe2\x80\x99s Tax Avoidance Is a Tax on the Rest of UsOpinion | Trump\xe2\x80\x99s Tax Avoidance Is a Tax on the Rest of UsOpinion | Trump\xe2\x80\x99s Tax Avoidance Is a Tax on the Rest of UsOpinion | Trump\xe2\x80\x99s Tax Avoidance Is a Tax on the Rest of UsOpinion | Trump\xe2\x80\x99s Tax Avoidance Is a Tax on the Rest of UsOpinion | Trump\xe2\x80\x99s Tax Avoidance Is a Tax on the Rest of UsOpinion | Trump\xe2\x80\x99

### Query 2

In [36]:
r2 = redis.Redis(host='my-redis', port=6379, db=0)

In [35]:
output2 = list(map(format_response, out2['response']['docs']))
output2

[{'title': 'Coronavirus tally: Global cases of COVID-19 35.5 million, 1.04 million deaths and U.S. death toll tops 210,000',
  'publishdate': '2020-10-06T00:00:00Z'},
 {'title': 'Coronavirus tally: Global cases of COVID-19 34.7 million, 1.03 million deaths and U.S. death toll nears 209,000',
  'publishdate': '2020-10-03T00:00:00Z'},
 {'title': 'Coronavirus tally: Global cases of COVID-19 38.2 million, 1.09 million deaths and U.S. death tolls nears 216,000',
  'publishdate': '2020-10-14T00:00:00Z'},
 {'title': 'Coronavirus tally: Global cases of COVID-19 40.4 million, 1.1 million deaths and U.S. death toll tops 220,000',
  'publishdate': '2020-10-20T00:00:00Z'},
 {'title': 'Coronavirus tally: Global cases of COVID-19 40.8 million, 1.13 million deaths and U.S. death toll tops 221,000',
  'publishdate': '2020-10-21T00:00:00Z'},
 {'title': 'Coronavirus tally: Global cases of COVID-19 41.28 million, 1.13 million deaths and U.S. death toll tops 222,000',
  'publishdate': '2020-10-22T00:00:00

In [42]:
for out in output2:
    r2.append(out['publishdate'], out['title'])

In [43]:
pipe = r2.pipeline()
pipe.get('2020-10-16T00:00:00Z')
pipe.execute()


[b'Coronavirus tally: Global cases of COVID-19 38.9 million, 1.09 million deaths and U.S. nears 8 million cases']

### Query 3

In [44]:
r3 = redis.Redis(host='my-redis', port=6379, db=0)

In [45]:
output3 = list(map(format_response, out3['response']['docs']))
output3

[{'title': 'PA: Biden 49% Trump 43%', 'publishdate': '2020-11-03T00:00:00Z'},
 {'title': 'TX: Trump 50% Biden 46%', 'publishdate': '2020-11-03T00:00:00Z'},
 {'title': 'MT: Trump 50% Biden 46%', 'publishdate': '2020-11-03T00:00:00Z'},
 {'title': 'WI: Biden 50% Trump 44%', 'publishdate': '2020-11-03T00:00:00Z'},
 {'title': 'A nation on edge, Biden and Trump race to the finish',
  'publishdate': '2020-11-03T00:00:00Z'},
 {'title': 'Last day: Trump rails, Biden pushes',
  'publishdate': '2020-11-03T00:00:00Z'},
 {'title': 'Last day: Trump rails, Biden pushes',
  'publishdate': '2020-11-03T00:00:00Z'},
 {'title': 'In 2020 finale, Trump combative, Biden on offense',
  'publishdate': '2020-11-03T00:00:00Z'},
 {'title': 'In 2020 finale, Trump combative, Biden on offense',
  'publishdate': '2020-11-03T00:00:00Z'},
 {'title': 'North Carolina: Biden 48% Trump 47%',
  'publishdate': '2020-11-03T00:00:00Z'}]

In [46]:
for out in output3:
    r3.append(out['publishdate'], out['title'])

In [47]:
pipe = r3.pipeline()
pipe.get('2020-11-03T00:00:00Z')
pipe.execute()

[b'PA: Biden 49% Trump 43%TX: Trump 50% Biden 46%MT: Trump 50% Biden 46%WI: Biden 50% Trump 44%A nation on edge, Biden and Trump race to the finishLast day: Trump rails, Biden pushesLast day: Trump rails, Biden pushesIn 2020 finale, Trump combative, Biden on offenseIn 2020 finale, Trump combative, Biden on offenseNorth Carolina: Biden 48% Trump 47%']